In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func

In [2]:
#read in json file with groceries data
#nrows=10 for testing code
Grocery_json = pd.read_json('Resources/Grocery_and_Gourmet_Food.json',lines=True, nrows=500)


In [3]:
Grocery_json

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,NaN,NaN
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,NaN,NaN
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,NaN,NaN
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,NaN,NaN
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
495,5,True,"10 13, 2015",A3AAPTA7ADEXC9,9742356831,One Fine Day,Great flavor! I purchased this after being fru...,Keep this one up your sleeve for a quick and f...,1444694400,NaN,NaN
496,5,True,"10 9, 2015",A69643EIL0ZDC,9742356831,David Miller,Good product.,Five Stars,1444348800,NaN,NaN
497,5,True,"09 28, 2015",A1T9SJTTRD2K5D,9742356831,Angela Waters,Love this curry! I used to live in Hawaii & th...,Love this curry,1443398400,NaN,NaN
498,4,True,"09 22, 2015",A1ECWZ09Y4I934,9742356831,Aijing Lin,"The taste is really great, similar to those cu...","The taste is really great, similar to those cu...",1442880000,NaN,NaN


In [4]:
#read labeled csv, will be used to count words in each review to generate review length column. 
labeled_df = pd.read_csv('Resources/labeled.csv')
labeled_df

,words,sentiment
0,"['comforting', 'know', 'processed', 'like', 's...",pos
1,"['never', 'disappointed', 'cup', 'cider', 'lik...",neg
2,"['never', 'ever', 'write', 'reviews', 'product...",neg
3,"['tasty', 'package', 'arrived', 'neatly', 'pac...",pos
4,"['gonna', 'buy', 'future', 'ate', 'completely'...",pos
...,...,...
809635,"['great', 'tasting', 'tuna', 'easy', 'open', '...",pos
809636,"['addicted', 'product', 'grocer', 'quit', 'car...",neg
809637,"['sure', 'happened', 'box', 'arrived', 'comple...",neg
809638,"['put', 'water', 'bottle', 'done', 'working', ...",pos


In [5]:
#count the number of words in a review and add a review length column
Grocery_json["reviewlength"] = labeled_df["words"].apply(lambda x: len(x))
Grocery_json.head(5)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,reviewlength
0,5,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,NaN,NaN,270
1,4,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,NaN,NaN,287
2,4,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,NaN,NaN,1156
3,5,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,NaN,NaN,126
4,4,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,NaN,NaN,92


In [6]:
#create groceries_df with the relevant columns
groceries_df = Grocery_json[['asin','overall','reviewerID', 'reviewTime','unixReviewTime', 'reviewText', "reviewlength"]]
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92
...,...,...,...,...,...,...,...
495,9742356831,5,A3AAPTA7ADEXC9,"10 13, 2015",1444694400,Great flavor! I purchased this after being fru...,8
496,9742356831,5,A69643EIL0ZDC,"10 9, 2015",1444348800,Good product.,263
497,9742356831,5,A1T9SJTTRD2K5D,"09 28, 2015",1443398400,Love this curry! I used to live in Hawaii & th...,390
498,9742356831,4,A1ECWZ09Y4I934,"09 22, 2015",1442880000,"The taste is really great, similar to those cu...",153


In [7]:
#drop dupicates (drops duplicates where the whole row is identical)
groceries_df = groceries_df.drop_duplicates()
groceries_df

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
1,1888861614,4,A1KPIZOCLB9FZ8,"05 23, 2014",1400803200,Very nicely crafted but too small. Am going to...,287
2,1888861614,4,A2W0FA06IYAYQE,"05 9, 2014",1399593600,still very pretty and well made...i am super p...,1156
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126
4,1888861614,4,A2VNHGJ59N4Z90,"04 16, 2014",1397606400,It was just what I want to put at the top of m...,92
...,...,...,...,...,...,...,...
495,9742356831,5,A3AAPTA7ADEXC9,"10 13, 2015",1444694400,Great flavor! I purchased this after being fru...,8
496,9742356831,5,A69643EIL0ZDC,"10 9, 2015",1444348800,Good product.,263
497,9742356831,5,A1T9SJTTRD2K5D,"09 28, 2015",1443398400,Love this curry! I used to live in Hawaii & th...,390
498,9742356831,4,A1ECWZ09Y4I934,"09 22, 2015",1442880000,"The taste is really great, similar to those cu...",153


In [8]:
#create df of 5 star ratings only
rating_5 = groceries_df.loc[groceries_df['overall'] == 5]
rating_5.head()

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
3,1888861614,5,A2PTZTCH2QUYBC,"04 20, 2014",1397952000,"I got this for our wedding cake, and it was ev...",126
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
8,1888861614,5,AGKW3A1RB1YGO,"10 2, 2013",1380672000,Perfect!!! Can not wait to use on my Wedding c...,57
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44


In [9]:
#create df with 1 star rating only
rating_1 = groceries_df.loc[groceries_df['overall'] == 1]
rating_1.head()

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
5,1888861614,1,ATQL0XOLZNHZ4,"02 21, 2014",1392940800,"The S is beautiful, however upon checkout it i...",97
22,4639725183,1,AE937MVR5J8OL,"12 10, 2016",1481328000,Tasted old and not individually bag,104
27,4639725183,1,A27LF8222OIRRN,"09 10, 2016",1473465600,Not the great taste everyone was talking about :(,40
37,4639725183,1,A34RH95REHOQFO,"09 25, 2015",1443139200,Terrible. What a waste of money. This is not t...,1391
69,4639725043,1,A3ELDSSPP9RZPG,"03 31, 2017",1490918400,The tea is so fine it leaves tea particles flo...,194


In [10]:
#choosing every second row of the five star rating for a total of 600,000 to reduce its size from around 3.5 million ratings to a number closer to the 1 star ratings. 
rating_5_trimmed = rating_5[:1200000:2]
rating_5_trimmed

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
6,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
9,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44
11,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257
13,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33
...,...,...,...,...,...,...,...
489,9742356831,5,A2KXDR2MTICVNI,"10 28, 2015",1445990400,Using Thai Kitchen's red and green curry paste...,628
492,9742356831,5,A1LC9OU0CWPBRN,"10 20, 2015",1445299200,Great flavour and a good amount of heat withou...,18
494,9742356831,5,A1TIGQEXUGWXR6,"10 13, 2015",1444694400,Love this. Moderately spicy when compared to ...,98
496,9742356831,5,A69643EIL0ZDC,"10 9, 2015",1444348800,Good product.,263


In [11]:
#another way to merge the data from 1 and 5 star ratings into one df
g1_5 = pd.merge(rating_5_trimmed, rating_1, how='outer')
g1_5

,asin,overall,reviewerID,reviewTime,unixReviewTime,reviewText,reviewlength
0,1888861614,5,ALP49FBWT4I7V,"06 4, 2013",1370304000,Very pleased with my purchase. Looks exactly l...,270
1,1888861614,5,A94ZG5CWN70E7,"12 1, 2013",1385856000,Omg.. The S was inexpensive and EXACTLY what w...,22
2,1888861614,5,A1B65IWLUVOUQB,"09 6, 2013",1378425600,This was exactly what we were looking for to m...,44
3,4639725183,5,A1QVBUH9E1V6I8,"11 19, 2014",1416355200,No adverse comment.,257
4,4639725183,5,A2CHH5U12THP2D,"11 3, 2014",1414972800,Best black tea in US.\n\nHighly recommend.\nI ...,33
...,...,...,...,...,...,...,...
208,9742356831,1,ANJMKELOZ1LKY,"05 31, 2017",1496188800,Where is my curry????? I got someone else's ma...,63
209,9742356831,1,A9X0YU2HAAI4T,"01 31, 2017",1485820800,I don't know much about authentic Thai coockin...,144
210,9742356831,1,A2FZMMOZXACZDR,"08 29, 2016",1472428800,It is too spicy and also very salty. I trashed...,131
211,9742356831,1,AS039MM100OWE,"05 30, 2016",1464566400,This doesn't taste much like green curry. The ...,84


In [26]:
# sqlalchemy's create engine preparing the ability to connect to a sqlite database. then with .connect connecting to the database. 
# sqlite:///groc15trimmed.sqlite
engine = create_engine('sqlite:///reviewdb.db', echo=True)
sqlite_connection = engine.connect()

In [27]:
#connecting to database and inputting info in from dataframe
sqlite_table = "groceriesALL"
ALLgroceries=g1_5.to_sql(sqlite_table, sqlite_connection, if_exists='replace')

2021-06-24 09:19:15,546 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesALL")
2021-06-24 09:19:15,548 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:19:15,551 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesALL")
2021-06-24 09:19:15,553 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:19:15,556 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-24 09:19:15,558 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:19:15,561 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("groceriesALL")
2021-06-24 09:19:15,563 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:19:15,569 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-24 09:19:15,571 INFO sqlalchemy.engine.Engine [raw sql] ('groceriesALL',)
2021-06-24 09:19:15,574 INFO sqlalchemy.engine.Engine PRAGMA main.foreig

In [28]:
MLgroceries_df = g1_5[['asin', 'overall', 'reviewText']]
MLgroceries_df

,asin,overall,reviewText
0,1888861614,5,Very pleased with my purchase. Looks exactly l...
1,1888861614,5,Omg.. The S was inexpensive and EXACTLY what w...
2,1888861614,5,This was exactly what we were looking for to m...
3,4639725183,5,No adverse comment.
4,4639725183,5,Best black tea in US.\n\nHighly recommend.\nI ...
...,...,...,...
208,9742356831,1,Where is my curry????? I got someone else's ma...
209,9742356831,1,I don't know much about authentic Thai coockin...
210,9742356831,1,It is too spicy and also very salty. I trashed...
211,9742356831,1,This doesn't taste much like green curry. The ...


In [29]:
#connecting to database and inputting info in from dataframe
sqlite_table_2 = "groceriesMLData"
MLgroceries=MLgroceries_df.to_sql(sqlite_table_2, sqlite_connection, if_exists='replace')

2021-06-24 09:19:20,625 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesMLData")
2021-06-24 09:19:20,628 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:19:20,633 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("groceriesMLData")
2021-06-24 09:19:20,636 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:19:20,641 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-06-24 09:19:20,644 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:19:20,647 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("groceriesMLData")
2021-06-24 09:19:20,649 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-24 09:19:20,652 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-24 09:19:20,653 INFO sqlalchemy.engine.Engine [raw sql] ('groceriesMLData',)
2021-06-24 09:19:20,656 INFO sqlalchemy.engine.Engine PRAGMA

In [ ]:
#select and show data in groceries ALL table
#engine.execute('SELECT * FROM groceriesALL').fetchall()

In [ ]:
#engine.execute('SELECT * FROM groceriesMLData').fetchall()